In [7]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.datasets import imdb
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

import import_notebook
from readWrite import ReadWrite
from vectorToDoc import VectorToDoc

In [13]:
tweetFile = "combineVectors.csv"
tweetClassFile = "combineVectorsResult.txt"

readWrite = ReadWrite()
xdata = readWrite.readFile(tweetFile)
ydata = readWrite.readFileClassifier(tweetClassFile)

x_train = xdata[:int(len(xdata) * 0.8) + 1]
y_train = ydata[:int(len(ydata) * 0.8) + 1]
x_test = xdata[int(len(xdata) * 0.8):-1]
y_test = ydata[int(len(ydata) * 0.8):-1]

print("x_train: {}, shape: {}".format(len(x_train), x_train.shape))
print("y_train: {}, shape: {}".format(len(y_train), y_train.shape))
print("x_test: {}, shape: {}".format(len(x_test), x_test.shape))
print("y_test: {}, shape: {}".format(len(y_test), y_test.shape))

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(ydata)
encoded = encoder.transform(ydata)
# convert integers to dummy variables (i.e. one hot encoded)
encoding = np_utils.to_categorical(encoded)

encoding_train = encoding[:int(len(xdata) * 0.8) + 1]
encoding_test = encoding[int(len(ydata) * 0.8):-1]
print("encoding_train shape: {}".format(encoding_train.shape))
print("encoding_test shape: {}".format(encoding_test.shape))

Read complete, total records: 1125
Read complete, total records: 1125
x_train: 901, shape: (901, 20)
y_train: 901, shape: (901,)
x_test: 224, shape: (224, 20)
y_test: 224, shape: (224,)
encoding_train shape: (901, 3)
encoding_test shape: (224, 3)


In [15]:
vocabSize = 2000
outputDim = 64
recordsSize = 20

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

model = Sequential()
model.add(Embedding(vocabSize, outputDim, input_length=recordsSize))
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, input_dim=(20,), activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 64)            128000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 303       
Total params: 194,303
Trainable params: 194,303
Non-trainable params: 0
_________________________________________________________________


In [33]:
batch_size = outputDim
epochs = 35

model.fit(x_train, encoding_train, validation_data=(x_test, encoding_test), epochs=epochs, batch_size=outputDim)
scores = model.evaluate(x_test, encoding_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 901 samples, validate on 224 samples
Epoch 1/35
901/901 [==============================] - 0s - loss: 0.0156 - acc: 0.9937 - val_loss: 0.6305 - val_acc: 0.8780
Epoch 2/35
901/901 [==============================] - 0s - loss: 0.0152 - acc: 0.9941 - val_loss: 0.6446 - val_acc: 0.8690
Epoch 3/35
901/901 [==============================] - 0s - loss: 0.0138 - acc: 0.9952 - val_loss: 0.7188 - val_acc: 0.8705
Epoch 4/35
901/901 [==============================] - 0s - loss: 0.0142 - acc: 0.9948 - val_loss: 0.8740 - val_acc: 0.8452
Epoch 5/35
901/901 [==============================] - 0s - loss: 0.0199 - acc: 0.9908 - val_loss: 0.5505 - val_acc: 0.8780
Epoch 6/35
901/901 [==============================] - 0s - loss: 0.0229 - acc: 0.9900 - val_loss: 0.8178 - val_acc: 0.8557
Epoch 7/35
901/901 [==============================] - 0s - loss: 0.0213 - acc: 0.9930 - val_loss: 0.6351 - val_acc: 0.8690
Epoch 8/35
901/901 [==============================] - 0s - loss: 0.0221 - acc: 0.9911 - val_l

In [34]:
vectorsArr2 = readWrite.readFile('docVectors3.csv')
oriRecords2 = readWrite.readOriFile('user_tweets3.txt')

classResult = []
predicted = model.predict_classes(vectorsArr2)
predicted = np.reshape(predicted, (predicted.size,))

print()
print(predicted.shape)

for idx, score in enumerate(predicted):    
    if(score >= 1): 
        classResult.append(oriRecords2[idx])       
        print(predicted[idx])
        print(oriRecords2[idx])        

Read complete, total records: 510
320/510 [=================>............] - ETA: 0s
(510,)
2
@SMRT_Singapore @ChannelNewsAsia EVERY WEEK SAME 💩just different days When r we goin to have one fcuking wk w/o any… https://t.co/7Fn26Oc5G0

1
@SBSTransit_Ltd is it too much to ask that your drivers be trained in driving? Nausea should not be a normal part of a short bus ride! 🤢

2
@SMRT_Singapore Just board train towards bukit batok from jurong east. Running normal.

1
@SMRT_Singapore please ACTIVATE YOUR SHUTTLE BUS STOP SAVING FKING MONEY

2
@SMRT_Singapore daily delays in red line 😑

2
@SMRT_Singapore Shit. Raining day. Sure delay at NSL with any kind of reasons.

2
@SMRT_Singapore red line red line!!!

2
@SMRT_Singapore The red line towards Marina Bay is stuck between bradell andToa payoh for past 15mins.This is very frustrating

1
@SMRT_Singapore @ChannelNewsAsia Again&gt;&lt;

1
@SMRT_Singapore Air con down as well in this current train. 3232

2
[Bus service]Bus services affected by ro